In [1]:
%matplotlib inline

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

plt.style.use('ggplot')

pd.__version__

u'0.17.0'

In [24]:
SSF_subsidies = pd.read_csv("SSF_subsidies.csv")
countries = pd.read_csv("Country_Data.csv")
catch = pd.read_csv("SAU_Catches.csv")
type_EEZ = pd.read_csv("Subtype_Fleet_EEZ.csv")
all_subsidies = pd.read_csv("Subsidies2009.csv")

In [25]:
SSF_subsidies.columns.values

array(['ID', 'Cnumber', 'RegionID', 'RegionName', 'Type', 'SubType',
       'Category', 'total_subsidies', 'NewData', 'Developed', 'HDI_2005',
       'Countries', 'SSFsubsidies_assessed_percent',
       'weighted_by_sectorsize', 'Data_group',
       'SSF_totalCatch_EEZ_percent', '%SSF_totalCatch_Country',
       'SSF_USD_1000', 'Comment', 'DocSource', 'SourceVisitDate', 'Ref',
       'URL', 'User', 'OldComment'], dtype=object)

In [30]:
#  merge original all subsidies data with to align with subsidy 
#  type and whether the EEZ catch data or the FLeet catch data apply, 
#  see type_EEZ data, this is in preparation to later merge with SAU_catch data 

subsidies_type = pd.merge(
    left=all_subsidies,
    right=type_EEZ,
    how='left'
)


#  now this needs to be merged with country data to have coordinates per country
#  and have each country data point assigned to a subregion (thanks to Andres data)

subsidies_type_countries = pd.merge(
    left=subsidies_type,
    right=countries,
    how='left'
)


#  merge with SSF catch from SAU_catch data

subsidies_type_countries_catch = pd.merge(
    left=subsidies_type_countries,
    right=catch, how='left'
)

#  calculating mean SSF catch per subregion, 
#  needed to use to fill gaps for SSF subsidies that have not been assessed

catch_countries = pd.merge(
    left=catch, 
    right=countries, 
    how='left'
)



In [31]:

catch_by_subregion = catch_countries.groupby(['Subregion']).sum()
catch_by_subregion['SSF_subregion_catch'] = (
    catch_by_subregion['SSF_Catch']/catch_by_subregion['Total_Catch']
)

catch_by_subregion = catch_by_subregion.reset_index()

SSF_subregion_catch = catch_by_subregion.loc[:,['Subregion','SSF_subregion_catch']]



#  merging mean catch with all subsidies data
subsidies_type_countries_catch_meancatch = pd.merge(
    left=subsidies_type_countries_catch,right=SSF_subregion_catch, how='left'
) 

#  preparing SSF subsidies by merging with country data
SSF_subsidies_countries = pd.merge(
    left=SSF_subsidies,right=countries, how='left')

#  from SSF subsidies assessed calculate mean per subregion and per type 
#  to fill gaps of countries that hvae not been assessed
SSF_by_subregion = SSF_subsidies_countries.groupby(
    ['Subregion','Type']).sum(
)

In [37]:

SSF_by_subregion['SSF_subsidies_subregion'] = (
    SSF_by_subregion['SSF_USD_1000']/SSF_by_subregion['total_subsidies']
)
SSF_by_subregion.to_csv('SSF_by_subregion.csv')

In [39]:
SSF_by_subregion = SSF_by_subregion.reset_index()
SSF_subsidies_subregion = SSF_by_subregion.loc[:,['Subregion','Type','SSF_subsidies_subregion']]


In [42]:

#  as not all subsidies types are covered by countires assessed, 
#  mean values for region are being calculated to replace the empty cells
SSF_by_region = SSF_subsidies_countries.groupby(
    ['RegionName','Type']).sum(
)
SSF_by_region['SSF_subsidies_region'] = (
    SSF_by_region['SSF_USD_1000']/SSF_by_region['total_subsidies']
)
SSF_by_region = SSF_by_region.reset_index()
SSF_subsidies_region = SSF_by_region.loc[
    :,['RegionName','Type','SSF_subsidies_region']
]


In [43]:

#  merge mean subsidies by type and subregion to all subsidies data
catch_subsidies_subregion = pd.merge(
    left=subsidies_type_countries_catch_meancatch,right=SSF_subsidies_subregion, how='left'
)


In [44]:

#  merge mean subsidies by type and region to all subsidies data that now have included mean subsidies per subregion
catch_subsidies_subregion_region = pd.merge(
    left=catch_subsidies_subregion,right=SSF_subsidies_region, how='left'
)
catch_subsidies_subregion_region

ID  Cnumber  RegionID RegionName   Countries Type  \
0        1        8         1     Europe     Albania   A1   
1        2        8         1     Europe     Albania   A2   
2        3        8         1     Europe     Albania   A3   
3        4        8         1     Europe     Albania   B1   
4        5        8         1     Europe     Albania   B2   
5        6        8         1     Europe     Albania   B3   
6        7        8         1     Europe     Albania   B4   
7        8        8         1     Europe     Albania   B5   
8        9        8         1     Europe     Albania   B6   
9       10        8         1     Europe     Albania   B7   
10      11        8         1     Europe     Albania   C1   
11      12        8         1     Europe     Albania   C2   
12      13        8         1     Europe     Albania   C3   
13      14       12         3     Africa     Algeria   A1   
14      15       12         3     Africa     Algeria   A2   
15      16       12         3     Africa     Algeria   A3   
16      17       12         3     Africa     Algeria   B1   
17      18       12         3     Africa     Algeria   B2   
18      19       12         3     Africa     Algeria   B3   
19      20       12         3     Africa     Algeria   B4   
20      21       12         3     Africa     Algeria   B5   
21      22       12         3     Africa     Algeria   B6   
22      23       12         3     Africa     Algeria   B7   
23      24       12         3     Africa     Algeria   C1   
24      25       12         3     Africa     Algeria   C2   
25      26       12         3     Africa     Algeria   C3   
26      27       24         3     Africa      Angola   A1   
27      28       24         3     Africa      Angola   A2   
28      29       24         3     Africa      Angola   A3   
29      30       24         3     Africa      Angola   B1   
...    ...      ...       ...        ...         ...  ...   
1933  1960      882         5    Oceania       Samoa   B7   
1934  1961      882         5    Oceania       Samoa   C1   
1935  1962      882         5    Oceania       Samoa   C2   
1936  1963      882         5    Oceania       Samoa   C3   
1937  1964      887         2       Asia       Yemen   A1   
1938  1965      887         2       Asia       Yemen   A2   
1939  1966      887         2       Asia       Yemen   A3   
1940  1967      887         2       Asia       Yemen   B1   
1941  1968      887         2       Asia       Yemen   B2   
1942  1969      887         2       Asia       Yemen   B3   
1943  1970      887         2       Asia       Yemen   B4   
1944  1971      887         2       Asia       Yemen   B5   
1945  1972      887         2       Asia       Yemen   B6   
1946  1973      887         2       Asia       Yemen   B7   
1947  1974      887         2       Asia       Yemen   C1   
1948  1975      887         2       Asia       Yemen   C2   
1949  1976      887         2       Asia       Yemen   C3   
1950  1977      891         1     Europe  Montenegro   A1   
1951  1978      891         1     Europe  Montenegro   A2   
1952  1979      891         1     Europe  Montenegro   A3   
1953  1980      891         1     Europe  Montenegro   B1   
1954  1981      891         1     Europe  Montenegro   B2   
1955  1982      891         1     Europe  Montenegro   B3   
1956  1983      891         1     Europe  Montenegro   B4   
1957  1984      891         1     Europe  Montenegro   B5   
1958  1985      891         1     Europe  Montenegro   B6   
1959  1986      891         1     Europe  Montenegro   B7   
1960  1987      891         1     Europe  Montenegro   C1   
1961  1988      891         1     Europe  Montenegro   C2   
1962  1989      891         1     Europe  Montenegro   C3   

                        SubType            Category  total_subsidies NewData  \
0              Fisheries mangt.          Beneficial       387.260735   False   
1                   Fishery R&D          Beneficial       198.557

In [ ]:


#  catch_subsidies_subregion_region['SSF_subsidies_all] = 
#  if catch_subsidies_subregion_region['total subsidies'] > 0  
#  and catch_subsidies_subregion_region['SSF_subsidies_subregion'] = NaN use 
#  value from catch_subsidies_subregion_region['SSF_subsidies_region']




In [ ]:


#  work in calc using this SSF_meancatch_meansubsidies_region csv file
#  fill gaps from mean subsidies per subregion with mean subsidies per region
#  create SSF_meancatch_meansubsidies_all.csv and upload as the new working data spreadsheet with all SSF subsidies
SSF_meancatch_meansubsidies_region.to_csv('SSF_meancatch_meansubsidies_region.csv')
SSF_meancatch_meansubsidies_all = pd.read_csv("SSF_meancatch_meansubsidies_all.csv")


In [ ]:
np.sum(SSF_meancatch_meansubsidies_all['ReEst_Subsidy2009_original'])

In [ ]:
#  create a new column 'Factor' and calculate as adjustment value for mean subsidies per subregion and mean SSF catch per subregion 
#  used to fill gaps for country that have not been assessed, this is done per subsidiey subtype also

SSF_meancatch_meansubsidies_all['Factor'] = SSF_meancatch_meansubsidies_all[
    'SSF_Catch_percent']/SSF_meancatch_meansubsidies_all['mean_catch'
                                                        ]

#  multiply the "Factor" value with the mean subsidy value per subregion
#  to estimate the value for SSF subsidies per subsidy type for the countries that have not been assessed
SSF_meancatch_meansubsidies_all['SSF_subsidies_estimates'] = SSF_meancatch_meansubsidies_all[
    'Factor']*SSF_meancatch_meansubsidies_all['mean_subsidies_subregion'
                                             ]
 
SSF_subsidies_short = SSF_subsidies_countries.loc[:,[
        'Countries','Type','SSFsubsidies_assessed_percent','weighted_by_sectorsize','Data_group','SSF_USD_1000']
                                                 ]

#  merge estimated values for subsidies with SSF assessed subsidies data
SSF_subsidies_estimates_assessed = pd.merge(
    left=SSF_meancatch_meansubsidies_all,right=SSF_subsidies_short,how='left'
)

#  use fillna function to use estimated values for SSF subsidies to fill gaps of countries that have not been assessed
SSF_subsidies_estimates_assessed['SSF_subsidies_percent'] = SSF_subsidies_estimates_assessed[
    'SSFsubsidies_assessed_percent'].fillna(
    SSF_subsidies_estimates_assessed['SSF_subsidies_estimates']
)

#  calculate USD values for all SSF for all subtypes and countries form the estimated and assessed percentages
SSF_subsidies_estimates_assessed['SSF_subsidies_USD'] = (
    SSF_subsidies_estimates_assessed['ReEst_Subsidy2009_original']/100
)*SSF_subsidies_estimates_assessed['SSF_subsidies_percent']

np.sum(SSF_subsidies_estimates_assessed['SSF_subsidies_USD'])


In [ ]:
#  save as csv file to add fuel subsidies
SSF_subsidies_estimates_assessed.to_csv('SSF_subsidies_estimates_assessed.csv')
#  read the newly created csv file now including fuel subsidy estimates
SSF_estimates_all = pd.read_csv("SSF_subsidies_withfuel.csv")

In [ ]:
np.sum(SSF_estimates_all['SSF_subsidies_USD'])

In [ ]:
subsidies_category = SSF_estimates_all.groupby(['Category']).sum()
subsidies_category.columns.values

In [ ]:
subsidies_categoryloc[:,[
        'Category','ReEst_Subsidy2009_original','SSF_USD_1000']

In [ ]:
Subsidies_per_Category['percent SSF Category'] = Subsidies_per_Category['SSF subsidies']/Subsidies_per_Category['total subsidies']*100

In [ ]:
 SSF_meancatch_meansubsidies_all